# El tratamiento de variables categoricas

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("../datasets/ecom-expense/Ecom Expense.csv")

In [3]:
df.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


Este ser aun analisis BigData como el de las empresas financeiras que nos comparan con personas dentro de nuestro rango de edad e ingresos.

las varaibles Gender y City Tier son **categoricas**, vamos a crear para cada una varaibles Dummy

In [4]:
dummy_gender = pd.get_dummies(df["Gender"], prefix="Gender")
dummy_city_tier = pd.get_dummies(df["City Tier"], prefix="City")
#esto dara una columna para cada valor y las llenara con 0´s y 1´s si aplica o no

In [5]:
dummy_gender.head()

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [6]:
dummy_city_tier.head()

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Ahora debemos **hacer un Join con el dataset principal**

In [7]:
column_names = df.columns.values.tolist() #tomo los nombres de las columnas y los guardo en una lista (array)
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend']

In [8]:
df_new = df[column_names].join(dummy_gender)#uno las columnas actuales con los datos dummy de genero
colum_names = df_new.columns.values.tolist()#guardo los nuevos nombres de columnas
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0


Ahora agregamos el valor de City

In [9]:
df_new = df_new[colum_names].join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


Lo siguiente es ver como integrar estos valores al modelo y como acceder a sus respectivos coeficientes. Vamos a obtener solo lo que necesitamos

In [10]:
feature_cols = ['Monthly Income','Transaction Time', 'Gender_Female', 
                'Gender_Male', 'City_Tier 1', 
                'City_Tier 2', 'City_Tier 3', 'Record'] #seran variables predictorias, 
#luego del primer intento se añadieron: 

In [11]:
X = df_new[feature_cols]#varaibles predictorias en x
Y = df_new["Total Spend"]#variable a predecir

creamos el modelo

In [12]:
lm = LinearRegression()#cargo objeto en la variable
lm.fit(X,Y)#inicio entrenamiento

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
lm.intercept_

-79.4171303013718

In [14]:
lm.coef_

array([ 1.47538980e-01,  1.54946125e-01, -1.31025013e+02,  1.31025013e+02,
        7.67643260e+01,  5.51389743e+01, -1.31903300e+02,  7.72233446e+02])

In [15]:
list(zip(feature_cols, lm.coef_))
#Gender Female y Gender male tienen los mismos valores (positivo y negativo)
#las City Tier si las sumamos dan 0

[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.15494612549589615),
 ('Gender_Female', -131.02501325554607),
 ('Gender_Male', 131.02501325554613),
 ('City_Tier 1', 76.76432601049524),
 ('City_Tier 2', 55.138974309232566),
 ('City_Tier 3', -131.9033003197279),
 ('Record', 772.2334457445644)]

In [16]:
lm.score(X,Y)#R2, ese valor 0.19478920552885381 es muy bajo, a este modelo aqui le falta informacion (sin el valor record)
#luego dio 0.9179923586131016 un R2 muy muy bueno

0.9179923586131016

**Conclusion:** haber añadido "Record" al modelo fue muy muy bueno

El modelo puede ser escrito como: (entre el valor de coef sea mas alto, mejor es la variable)

* Total_Spend = -79.41713030137362 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545+'Gender_Female' -131.02501325554567 + 'Gender_Male' 131.0250132555456+'City_Tier 1' 76.76432601049527 + 'City_Tier 2' 55.138974309232474 + 'City_Tier 3' -131.9033003197278+'Record' 772.2334457445648

    * Si es hombre y vive en CT1: Total_Spend = 128.37220896466724 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545+'Record'* 772.2334457445648
    
    * Si es hombre y vive en CT2: Total_Spend = 106.74685726340445 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 +'Record'* 772.2334457445648
    
    * Si es hombre y vive en CT3: Total_Spend = -80.29541736555583 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545+'Record'* 772.2334457445648
    
    * Si es mujer y vive en CT1: Total_Spend = -79.41713030137362 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 - 131.0250132555456+ 76.76432601049527 +'Record'* 772.2334457445648
    
    * Si es mujer y vive en CT2: Total_Spend = -79.41713030137362 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 - 131.0250132555456+ 55.138974309232474 +'Record'* 772.2334457445648
    
    * Si es mujer y vive en CT3: Total_Spend = -79.41713030137362 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 - 131.0250132555456-131.9033003197278 +'Record'* 772.2334457445648


In [17]:
#agregamos la prediccion a nuestro dataset (PERO PODEMOS HACER ESTO USANDO LA FUNCION PREDICT)
df_new["prediction"] = -79.41713030137362 + df_new['Monthly Income']*0.14753898049205738 + df_new['Transaction Time']* 0.15494612549589545+ df_new['Gender_Female'] * (-131.02501325554567) + df_new['Gender_Male'] * 131.0250132555456+ df_new['City_Tier 1']* 76.76432601049527 +  df_new['City_Tier 2']* 55.138974309232474 + df_new['City_Tier 3']* (-131.9033003197278)+ df_new['Record']* 772.2334457445648

In [18]:
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,4903.696720
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4799.434826
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,5157.082504
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,8068.012996
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3581.980335


In [19]:
SSD = np.sum((df_new["prediction"] - df_new["Total Spend"])**2)
SSD

1517733985.3408163

In [20]:
RSE = np.sqrt(SSD/(len(df_new)-len(feature_cols)-1))
RSE

803.1318809818165

In [21]:
sales_mean=np.mean(df_new["Total Spend"])
sales_mean

6163.176415976714

In [22]:
error = RSE/sales_mean
error*100 #100 para hacerlo en porcentaje

13.031135680294161

Aunque el error es de 13% (que no es tan bajito) es aceptable por que tenemos muchas varaibles y muchisimas filas